# EEC 174AY Lab B2

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from IPython.core.display import HTML
HTML("""
<style>
.text_cell_render p{
    font-size: 130%;
    line-height: 125%;
}
</style>
""")

## Outline

This lab will build your skills in utilizing LSTM networks so that you can apply deep learning to time series information

1. you will code an LSTM network and apply it to a pre-built codebase. Your focus will be on the ML coding
2. You will utilize a partially built code base and then finish it to detect ARDS in waveform data.

## LSTM Network

LSTM is a network that is able to utilize time series information and learn long term patterns to make more accurate predictions than a normal neural network would be able to. We show the general network architecture as an instruction for what you will need to code.

# <img src="/content/drive/MyDrive/Colab Notebooks/Lab_B2/The_LSTM_cell.png" width=55% height=auto\>

You will be applying LSTM to the task of patient ventilator asynchrony (PVA) detection. We have supplied a bit of the code you will need. Your jobs will be the following:

1. Code the `find_scaling_coefs`, `scale_breath`, and `pad_or_cut_breath` methods in the `PVADataset` class in `dataset.py`.
2. Code a simple 1 layer LSTM network based on network schematics given above. You are welcome to use other resource for assistance as well.
3. Run your LSTM model on PVA detection. How well does your model perform compared to your original Random Forest classifier? Why are you getting these results?
4. Code a more complex 3 layer LSTM network. Do additional layers improve results? Why/Why not?

For the math required we would advise you follow the [PyTorch LSTM mathematics](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)

In [5]:
from torch import nn
from torch.nn import functional as F

# class LSTMNetwork(nn.Module):
#     def __init__(self):
#         super(LSTMNetwork, self).__init__()
#         lstm_hidden_units = 32
#         self.lstm = nn.LSTM(input_size=2, hidden_size=lstm_hidden_units, batch_first=True)
#         self.final_classification = nn.Linear(lstm_hidden_units, 3)

#     def forward(self, x):
#         h0 = torch.zeros(1, x.size(0), self.lstm.hidden_size).to(x.device)
#         c0 = torch.zeros(1, x.size(0), self.lstm.hidden_size).to(x.device)
#         out, _ = self.lstm(x, (h0, c0))
#         out = self.final_classification(out[:, -1, :])
#         return out


class LSTMNetwork(nn.Module):
    def __init__(self):
        super(LSTMNetwork, self).__init__()
        # Feel free to modify this
        lstm_hidden_units = 32
        # First layer is defined for you. Only have 2 input features (flow, pressure)
        self.ii = nn.Linear(2, lstm_hidden_units)
        # XXX TODO
        self.dummy = nn.Linear(2, lstm_hidden_units)
        self.s2 = nn.Linear(2, lstm_hidden_units)
        self.s3 = nn.Linear(2, lstm_hidden_units)
        self.t1 = nn.Linear(2, lstm_hidden_units)
        # Final layer is defined for you too. Have 3 potential output classes (normal, bsa, dta)
        self.final_classification = nn.Linear(lstm_hidden_units, 3)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()

    def forward(self, x):
        # XXX code this up
        trial = self.dummy(x[:, 1, :])
        trial = self.sigmoid(trial)
        ct_next = torch.zeros_like(trial)
        for i in range(x.size(1)):
          r = x[:, i, :]
          ct_prev = ct_next
          sig1 = self.ii(r)
          sig1 = self.sigmoid(sig1)
          sig2 = self.s2(r)
          sig2 = self.sigmoid(sig2)
          sig3 = self.s3(r)
          sig3 = self.sigmoid(sig3)
          tan1 = self.t1(r)
          tan1 = self.tanh(tan1)
          ct_s1 = ct_prev * sig1
          s2_t1 = sig2 * tan1
          ct_next = ct_s1 + s2_t1
          ct_tan = self.tanh(ct_next)
          out = sig3 * ct_tan
          out = self.final_classification(out)
        return out

In [6]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
from glob import glob
import torch
from torch import nn
from torch.autograd import Variable
from torch.optim import SGD
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score

class PVADataset(Dataset):
    def __init__(self, dataset_type, sequence_len):
        """
        Extract breath data and corresponding PVA annotations

        :param dataset_type: What set we are using (train/val/test)
        :param sequence_len: The length of the sequences we want to give to our LSTM
        """
        if dataset_type not in ['train', 'val', 'test']:
            raise Exception('dataset_type must be either "train", "val", or "test"')

        dataset_path = os.path.join('/content/drive/MyDrive/Colab_Notebooks/Lab_B2', 'pva_dataset', dataset_type)
        self.record_set = glob(os.path.join(dataset_path, '*_data.pkl'))
        self.all_sequences = []
        self.sequence_len = sequence_len

    def process_dataset(self):
        """
        Extract all breaths in the dataset and pair a ground truth value
        with the breath information
        """
        for record in self.record_set:
            data = pd.read_pickle(record)
            gt = pd.read_pickle(record.replace('_data.pkl', '_gt.pkl'))
            patient = os.path.basename(record).split('_')[0]
            for i, b in enumerate(data):
                gt_row = gt.iloc[i]
                if gt_row.bn != b['rel_bn']:
                    raise Exception('something went wrong with gt parsing for record {}'.format(record))
                if gt_row.dta >= 1:
                    y = [0, 0, 1]
                elif gt_row.bsa >= 1:
                    y = [0, 1, 0]
                else:
                    y = [1, 0, 0]

                tensor = np.array([b['flow'], b['pressure']]).transpose()
                self.all_sequences.append([patient, tensor, np.array(y)])

        self.find_scaling_coefs()

    def find_scaling_coefs(self):
        """
        In order to conduct scaling you will need to find some scaling
        coefficients that are represented in our data. The time to find
        these coefficients is right after the data has been processed
        into a machine-usable format
        """
        # write function for finding the scaling coefficients
        all_data = np.concatenate([item[1] for item in self.all_sequences], axis=0)
        # pressure_and_flow_values = data_array[:, :2]  # Extracting the first two columns (pressure and flow)

        # Calculate mean and standard deviation for pressure and flow
        # mean_values = np.mean(all_data, axis=0)
        # std_values = np.std(all_data, axis=0)

        # print("Mean Values (Pressure, Flow):", mean_values)
        # print("Standard Deviation Values (Pressure, Flow):", std_values)
        # print(self.all_sequences[:2])
        # print(all_data[:5])
        self.scale_breath(all_data)

    def scale_breath(self, data):
        """
        Scale breath using any number of scaling techniques learned in
        this class. You can use standardization, max-min scaling, or
        anything else that you'd like to code
        """
        # mean = self.scaling_coefficients['mean']
        # print(mean.shape())
        # std  = self.scaling_coefficients['std']
        # print(std.shape())
        mean_values = np.mean(data, axis=0)
        std_values = np.std(data, axis=0)
        output = (data - mean_values) / std_values
        # print(data)
        # print(output)
        return output

    def pad_or_cut_breath(self, data):
        """
        For purposes of the simple LSTM that you are going to code you
        will need to have all your breaths be of uniform size. This means
        adding a padded value like 0 to a sequence to ensure a breath reaches
        desired length. It could also mean removing observations from a
        sequence if the data is longer than desired length
        """
        desired_length = self.sequence_len
        current_length = len(data)

        if current_length < desired_length:
            # Pad with zeros to reach the desired length
            padding = np.zeros((desired_length - current_length, data.shape[1]))
            data = np.concatenate([data, padding], axis=0)
        elif current_length > desired_length:
            # Truncate to the desired length
            data = data[:desired_length, :]
            # print(data.shape())
        return data

    def __getitem__(self, idx):
        """
        get next sequence
        """
        pt, data, y = self.all_sequences[idx]
        data = self.scale_breath(data)
        data = self.pad_or_cut_breath(data)
        return data, y

    def __len__(self):
        return len(self.all_sequences)

model = LSTMNetwork().cuda()
# You should modify the learning rate as suits the problem
optimizer = SGD(model.parameters(), lr=0.01)
bce = nn.BCEWithLogitsLoss()
batch_size = 16


def get_dataset(path, name):
    saved_set = Path(path)
    # Make sure we save previously processed data. This speeds up future processes.
    if saved_set.exists():
        dataset = pd.read_pickle(saved_set.resolve())
    else:
        # use a sequence length of 224 inputs. If you want to shorten this feel free.
        dataset = PVADataset(name, 224)
        dataset.process_dataset()
        pd.to_pickle(dataset, saved_set.resolve())
    return dataset

def get_all_datasets():
    training_set = get_dataset('pva_training_set.pkl', 'train')
    validation_set = get_dataset('pva_validation_set.pkl', 'val')
    testing_set = get_dataset('pva_testing_set.pkl', 'test')
    return training_set, validation_set, testing_set

def perform_training_epoch(train_loader):
    model.train()
    train_loss = 0.0
    train_corrects = 0
    train_total = 0

    for x, y in train_loader:
        x = x.float().cuda()
        y = y.float().cuda()
        optimizer.zero_grad()  # Zero the gradients

        output = model(x)
        loss = bce(output, y)
        loss.backward()
        optimizer.step()

        # Update training loss
        train_loss += loss.item() * x.size(0)

        # Calculate training accuracy
        binary_predictions = (output > 0.5).float()
        train_corrects += torch.sum(binary_predictions == y.data)
        train_total += y.size(0)

    # Calculate training accuracy
    train_accuracy = train_corrects / train_total
    train_loss /= len(train_loader.dataset)

    print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")

    return train_loss, train_accuracy

def perform_inferencing(loader):
    model.eval()
    all_predictions = []
    all_targets = []
    total_loss = 0.0
    total_corrects = 0
    total_samples = 0

    with torch.no_grad():
        for x, y in loader:
            x = x.float().cuda()

            output = model(x)

            # Calculate loss
            loss = bce(output, y.float().cuda())
            total_loss += loss.item() * x.size(0)

            # Store predictions and ground truth labels
            all_predictions.append(output.cpu().numpy())
            all_targets.append(y.cpu().numpy())

            # Calculate accuracy
            binary_predictions = (output > 0.5).cpu().float()  # Move to CPU
            total_corrects += torch.sum(binary_predictions == y.data.cpu())
            total_samples += y.size(0)

    # Concatenate predictions and ground truth labels
    predictions = np.concatenate(all_predictions)
    targets = np.concatenate(all_targets)

    # Convert probabilities to binary predictions
    binary_predictions = (predictions > 0.5).astype(int)

    # Calculate F1 score
    f1 = f1_score(targets, binary_predictions, average='micro')

    # Calculate accuracy
    accuracy = total_corrects / total_samples

    # Calculate average loss
    average_loss = total_loss / len(loader.dataset)

    print(f"F1 Score: {f1:.4f}")
    return average_loss, accuracy

training_set, validation_set, testing_set = get_all_datasets()
# XXX make sure val and testing share same coefficients as training set!!

train_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validation_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(testing_set, batch_size=batch_size, shuffle=False)
# You can write up the rest of the code here. We have already given you most of
# what you need to run the module yourself.

num_epochs = 5
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Training
    perform_training_epoch(train_loader)

    # Validation
    print("Validation Results")
    validation_loss, validation_accuracy = perform_inferencing(val_loader)
    print(f"Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_accuracy:.4f}")

# Test the model on the testing set
print("Testing Results")
test_loss, test_accuracy = perform_inferencing(test_loader)
print(f"Test Accuracy: {test_accuracy:.4f}")

Epoch 1/5
Training Loss: 0.5999, Training Accuracy: 2.0000
Validation Results
F1 Score: 0.0020
Validation Loss: 0.5794, Validation Accuracy: 2.0003
Epoch 2/5
Training Loss: 0.5254, Training Accuracy: 2.1912
Validation Results
F1 Score: 0.5700
Validation Loss: 0.5620, Validation Accuracy: 2.1400
Epoch 3/5
Training Loss: 0.5076, Training Accuracy: 2.3090
Validation Results
F1 Score: 0.5700
Validation Loss: 0.5603, Validation Accuracy: 2.1400
Epoch 4/5
Training Loss: 0.5019, Training Accuracy: 2.3090
Validation Results
F1 Score: 0.5700
Validation Loss: 0.5609, Validation Accuracy: 2.1400
Epoch 5/5
Training Loss: 0.4998, Training Accuracy: 2.3090
Validation Results
F1 Score: 0.5700
Validation Loss: 0.5619, Validation Accuracy: 2.1400
Testing Results
F1 Score: 0.4960
Test Accuracy: 1.9920


## ARDS Detection

Regardless of whether you were successful on your last assignment, the design was to show you the internal mechanism about how LSTM works.

In this assignment you will utilize a dataset of ventilation data taken from 50 subjects. 25 subjects have ARDS, 25 subjects do not have ARDS. Your job is to extract waveform data, and utilize it to perform inferencing on whether the patient has ARDS or not.

1. Use basic CNN architecture to perform classification on whether patient has ARDS or not
2. Add LSTM to CNN architecture, do results improve? if not why? In this assignment you should use the [PyTorch LSTM layer.](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)

### Data

The data that we use here is ventilation data but it is structured a bit differently than the PVA dataset. Primarily, the data is structured in continuous breath sequences instead of single breaths. Here is an example.

<img src=ards-data.png width=50% height=auto\>

This has a few advantages:

1. We don't need padding anymore
2. It improves performance of our model

We stack 20 of these breaths together into a tensor that is in shape `(20, 1, 224)`. This allows us to analyze sequential breaths with an LSTM if we desire.

In [3]:
pip install ventmap

  Preparing metadata (setup.py) ... done
  Created wheel for ventmap: filename=ventmap-1.5.3-py3-none-any.whl size=40312 sha256=ee0bd851f144d690f2d0cc6301a0edaafe821f5257f4bd34276f80f4482e22fd
  Stored in directory: /root/.cache/pip/wheels/6b/18/ac/0abd36110fb734afe3ba7c3e4a69a2c14f8022ee77ba30db13
Successfully built ventmap


In [7]:
from pathlib import Path
from copy import copy
from glob import glob
import math
import os
import re

import numpy as np
import pandas as pd
from scipy.signal import resample
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader
from ventmap.raw_utils import read_processed_file

class ARDSDataset(Dataset):
    def __init__(self, seq_len, dataset_type, to_pickle=None):
        """
        Dataset to generate sequences of data for ARDS Detection
        """
        data_path = os.path.join('/content/drive/MyDrive/Colab_Notebooks/EEC174LabB2', 'data')
        cohort_file = os.path.join(data_path, 'anon-desc.csv')
        self.seq_len = seq_len
        self.n_sub_batches = 20
        self.all_sequences = []

        self.cohort = pd.read_csv(cohort_file)
        self.cohort = self.cohort.rename(columns={'Patient Unique Identifier': 'patient_id'})
        self.cohort['patient_id'] = self.cohort['patient_id'].astype(str)

        raw_dir = os.path.join(data_path, 'experiment1', dataset_type, 'raw')
        if not os.path.exists(raw_dir):
            raise Exception('No directory {} exists!'.format(raw_dir))
        self.raw_files = sorted(glob(os.path.join(raw_dir, '*/*.raw.npy')))
        self.processed_files = sorted(glob(os.path.join(raw_dir, '*/*.processed.npy')))
        self.get_dataset()
        self.derive_scaling_factors()
        if to_pickle:
            pd.to_pickle(self, to_pickle)

    def derive_scaling_factors(self):
        indices = [range(len(self.all_sequences))]
        self.scaling_factors = {
            None: self._get_scaling_factors_for_indices(idxs)
            for i, idxs in enumerate(indices)
        }

    @classmethod
    def from_pickle(self, data_path):
        dataset = pd.read_pickle(data_path)
        if not isinstance(dataset, ARDSDataset):
            raise ValueError('The pickle file you have specified is out-of-date. Please re-process your dataset and save the new pickled dataset.')
        # paranoia
        try:
            dataset.scaling_factors
        except AttributeError:
            dataset.derive_scaling_factors()
        return dataset

    def get_dataset(self):
        last_patient = None
        for fidx, filename in enumerate(self.raw_files):
            gen = read_processed_file(filename, filename.replace('.raw.npy', '.processed.npy'))
            patient_id = self._get_patient_id_from_file(filename)

            if patient_id != last_patient:
                batch_arr = []
                breath_arr = []
                seq_vent_bns = []
                batch_seq_hours = []

            last_patient = patient_id
            target = self._pathophysiology_target(patient_id)
            start_time = self._get_patient_start_time(patient_id)

            for bidx, breath in enumerate(gen):
                # cutoff breaths if they have too few points.
                if len(breath['flow']) < 21:
                    continue

                breath_time = self.get_abs_bs_dt(breath)
                if breath_time < start_time:
                    continue
                elif breath_time > start_time + pd.Timedelta(hours=24):
                    break

                flow = breath['flow']
                seq_hour = (breath_time - start_time).total_seconds() / 60 / 60
                seq_vent_bns.append(breath['vent_bn'])
                batch_arr, breath_arr, batch_seq_hours = self._unpadded_centered_processing(
                    flow, breath_arr, batch_arr, batch_seq_hours, seq_hour
                )

                if len(batch_arr) == self.n_sub_batches:
                    raw_data = np.array(batch_arr)
                    breath_window = raw_data.reshape((self.n_sub_batches, 1, self.seq_len))
                    self.all_sequences.append([patient_id, breath_window, target, batch_seq_hours])
                    batch_arr = []
                    seq_vent_bns = []
                    batch_seq_hours = []

                if len(batch_arr) > 0 and breath_arr == []:
                    batch_seq_hours.append(seq_hour)

    def get_abs_bs_dt(self, breath):
        if isinstance(breath['abs_bs'], bytes):
            breath['abs_bs'] = breath['abs_bs'].decode('utf-8')
        try:
            breath_time = pd.to_datetime(breath['abs_bs'], format='%Y-%m-%d %H-%M-%S.%f')
        except:
            breath_time = pd.to_datetime(breath['abs_bs'], format='%Y-%m-%d %H:%M:%S.%f')
        return breath_time

    def _pathophysiology_target(self, patient_id):
        patient_row = self.cohort[self.cohort['patient_id'] == patient_id]
        try:
            patient_row = patient_row.iloc[0]
        except:
            raise ValueError('Could not find patient {} in cohort file'.format(patient_id))
        patho = 1 if patient_row['Pathophysiology'] == 'ARDS' else 0
        target = np.zeros(2)
        target[patho] = 1
        return target

    def _unpadded_centered_processing(self, flow, breath_arr, batch_arr, batch_seq_hours, seq_hour):
        if (len(flow) + len(breath_arr)) < self.seq_len:
            breath_arr.extend(flow)
        else:
            remaining = self.seq_len - len(breath_arr)
            breath_arr.extend(flow[:remaining])
            batch_arr.append(np.array(breath_arr))
            batch_seq_hours.append(seq_hour)
            breath_arr = []
        return batch_arr, breath_arr, batch_seq_hours

    def _get_scaling_factors_for_indices(self, indices):
        """
        Get mu and std for a specific set of indices
        """
        std_sum = 0
        mean_sum = 0
        obs_count = 0

        for idx in indices:
            obs = self.all_sequences[idx][1]
            obs_count += len(obs)
            mean_sum += obs.sum()
        mu = mean_sum / obs_count

        # calculate std
        for idx in indices:
            obs = self.all_sequences[idx][1]
            std_sum += ((obs - mu) ** 2).sum()
        std = np.sqrt(std_sum / obs_count)
        return mu, std

    def _pathophysiology_target(self, patient_id):
        patient_row = self.cohort[self.cohort['patient_id'] == patient_id]
        try:
            patient_row = patient_row.iloc[0]
        except:
            raise ValueError('Could not find patient {} in cohort file'.format(patient_id))
        patho = 1 if patient_row['Pathophysiology'] == 'ARDS' else 0
        target = np.zeros(2)
        target[patho] = 1
        return target

    def _get_patient_start_time(self, patient_id):
        patient_row = self.cohort[self.cohort['patient_id'] == patient_id]
        patient_row = patient_row.iloc[0]
        patho = 1 if patient_row['Pathophysiology'] == 'ARDS' else 0
        if patho == 1:
            start_time = pd.to_datetime(patient_row['Date when Berlin criteria first met (m/dd/yyy)'])
        else:
            start_time = pd.to_datetime(patient_row['vent_start_time'])

        if start_time is pd.NaT:
            raise Exception('Could not find valid start time for {}'.format(patient_id))
        return start_time

    def __getitem__(self, index):
        seq = self.all_sequences[index]
        pt, data, target, _ = seq
        try:
            mu, std = self.scaling_factors[None]
        except AttributeError:
            raise AttributeError('Scaling factors not found for dataset. You must derive them using the `derive_scaling_factors` function.')
        data = (data - mu) / std

        return pt, data, target

    def __len__(self):
        return len(self.all_sequences)

    def get_ground_truth_df(self):
        return self._get_all_sequence_ground_truth()

    def _get_all_sequence_ground_truth(self):
        rows = []
        for seq in self.all_sequences:
            patient, _, target = seq
            rows.append([patient, np.argmax(target, axis=0)])
        return pd.DataFrame(rows, columns=['patient', 'y'])

    def _get_patient_id_from_file(self, filename):
        pt_id = filename.split('/')[-2]
        # sanity check to see if patient
        match = re.search(r'(0\d{3}RPI\d{10})', filename)
        if match:
            return match.groups()[0]
        try:
            # id is from anonymous dataset
            float(pt_id)
            return pt_id
        except:
            raise ValueError('could not find patient id in file: {}'.format(filename))


batch_size = 32


def get_dataset(path, name):
    saved_set = Path(path)
    # Make sure we save previously processed data. This speeds up future processes.
    if saved_set.exists():
        dataset = ARDSDataset.from_pickle(saved_set.resolve())
    else:
        dataset = ARDSDataset(224, name, to_pickle=saved_set.resolve())
    return dataset


def get_all_datasets():
    training_set = get_dataset('ards_training_set.pkl', 'train')
    validation_set = get_dataset('ards_validation_set.pkl', 'val')
    testing_set = get_dataset('ards_testing_set.pkl', 'test')
    return training_set, validation_set, testing_set


training_set, validation_set, testing_set = get_all_datasets()
train_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validation_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(testing_set, batch_size=batch_size, shuffle=False)


In [11]:
from torch import nn
from torch.optim import SGD
import torch
from torch.autograd import Variable
import torch.nn as nn
import re
import torch.nn.functional as F
import math
from collections import OrderedDict

class CNNLSTMNetwork(nn.Module):
    def __init__(self, cnn_network, lstm_hidden_units, lstm_layers, lstm_dropout):
        super(CNNLSTMNetwork, self).__init__()

        self.cnn = cnn_network
        self.lstm_hidden_units = lstm_hidden_units
        # Add LSTM layer
        self.lstm = nn.LSTM(input_size=lstm_hidden_units,
                            hidden_size=lstm_hidden_units,
                            num_layers=lstm_layers,
                            dropout=lstm_dropout,
                            batch_first=True)

        # Add linear layer
        self.fc = nn.Linear(lstm_hidden_units, 2)

    def forward(self, x, hx_cx=None):
        # input should be in shape: (batches, breaths in seq, chans, 224)
        batches, seq_len, chans, _ = x.shape

        # Initialize an empty list to store outputs
        outputs = []

        # Process the first input breath separately
        cnn_out = self.cnn(x[:, 0, 0, :].unsqueeze(1)).squeeze()
        lstm_out, _ = self.lstm(cnn_out.unsqueeze(1), hx_cx)
        lstm_out = lstm_out[:, -1, :]
        output = self.fc(lstm_out)
        outputs.append(output)

        # Process the remaining input breaths
        intermediate_outputs = []  # List to store intermediate tensors
        for i in range(1, seq_len):
            cnn_out = self.cnn(x[:, i, 0, :].unsqueeze(1)).squeeze()
            lstm_out, _ = self.lstm(cnn_out.unsqueeze(1))
            lstm_out = lstm_out[:, -1, :]
            output = self.fc(lstm_out)
            intermediate_outputs.append(output)

        # Concatenate the intermediate tensors along the sequence dimension
        outputs = torch.cat([outputs[0]] + intermediate_outputs, dim=0)

        return output
    # def forward(self, x, hx_cx=None):
    #     # input should be in shape: (batches, breaths in seq, chans, 224)
    #     batches, seq_len, chans, _ = x.shape

    #     # Initialize an empty list to store intermediate LSTM outputs
    #     intermediate_outputs = []

    #     # Process each input breath
    #     for i in range(seq_len):
    #         cnn_out = self.cnn(x[:, i, 0, :].unsqueeze(1)).squeeze()

    #         # Use hidden and cell states for the LSTM layer
    #         lstm_out, (hidden_state, cell_state) = self.lstm(cnn_out.unsqueeze(1), hx_cx)
    #         lstm_out = lstm_out[:, -1, :]

    #         # Store intermediate LSTM outputs
    #         intermediate_outputs.append(lstm_out)

    #     # Concatenate the intermediate LSTM outputs along the sequence dimension
    #     intermediate_outputs = torch.cat(intermediate_outputs, dim=0)

    #     # Apply the fully connected layer to the concatenated intermediate LSTM outputs
    #     output = self.fc(intermediate_outputs)

    #     return output

__all__ = ['DenseNet', 'densenet121', 'densenet169', 'densenet201', 'densenet161']

model_urls = {
    'densenet121': 'https://download.pytorch.org/models/densenet121-a639ec97.pth',
    'densenet169': 'https://download.pytorch.org/models/densenet169-b2777c0a.pth',
    'densenet201': 'https://download.pytorch.org/models/densenet201-c1103571.pth',
    'densenet161': 'https://download.pytorch.org/models/densenet161-8d451a50.pth',
}


class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm1d(num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv1d(num_input_features, bn_size *
                                           growth_rate, kernel_size=1, stride=1,
                                           bias=False)),
        self.add_module('norm2', nn.BatchNorm1d(bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv1d(bn_size * growth_rate, growth_rate,
                                           kernel_size=3, stride=1, padding=1,
                                           bias=False)),
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate,
                                     training=self.training)
        return torch.cat([x, new_features], 1)


class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate, growth_rate,
                                bn_size, drop_rate)
            self.add_module('denselayer%d' % (i + 1), layer)


class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm1d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv1d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool1d(kernel_size=2, stride=2))


class DenseNet(nn.Module):
    r"""Densenet-BC model class, based on
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        growth_rate (int) - how many filters to add each layer (`k` in paper)
        block_config (list of 4 ints) - how many layers in each pooling block
        num_init_features (int) - the number of filters to learn in the first convolution layer
        bn_size (int) - multiplicative factor for number of bottle neck layers
          (i.e. bn_size * k features in the bottleneck layer)
        drop_rate (float) - dropout rate after each dense layer
        num_classes (int) - number of classification classes
    """

    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0.2, num_classes=1000):

        super(DenseNet, self).__init__()
        self.inplanes = num_init_features

        # First convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv1d(1, num_init_features, kernel_size=7, stride=2,
                                padding=3, bias=False)),
            ('norm0', nn.BatchNorm1d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool1d(kernel_size=3, stride=2, padding=1)),
        ]))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers, num_input_features=num_features,
                                bn_size=bn_size, growth_rate=growth_rate,
                                drop_rate=drop_rate)
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features,
                                    num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm1d(num_features))

        # Linear layer
        #self.classifier = nn.Linear(num_features, num_classes)

        # Official init from torch repo.
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                n = m.kernel_size[0] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

        self.n_out_filters = num_features
        self.avgpool = nn.AvgPool1d(7, stride=1)

    def forward(self, x):
        features = self.features(x)
        #print(features.size())
        out = F.relu(features, inplace=True)
        out = self.avgpool(out)
        out = out.view(features.size(0), -1)
        #out = F.adaptive_avg_pool2d(out, (1, 1)).view(features.size(0), -1)
        #out = self.classifier(out)
        #print(out.size())
        #print(self.n_out_filters)
        return out


def _load_state_dict(model, model_url, progress):
    # '.'s are no longer allowed in module names, but previous _DenseLayer
    # has keys 'norm.1', 'relu.1', 'conv.1', 'norm.2', 'relu.2', 'conv.2'.
    # They are also in the checkpoints in model_urls. This pattern is used
    # to find such keys.
    pattern = re.compile(
        r'^(.*denselayer\d+\.(?:norm|relu|conv))\.((?:[12])\.(?:weight|bias|running_mean|running_var))$')

    state_dict = load_state_dict_from_url(model_url, progress=progress)
    for key in list(state_dict.keys()):
        res = pattern.match(key)
        if res:
            new_key = res.group(1) + res.group(2)
            state_dict[new_key] = state_dict[key]
            del state_dict[key]
    model.load_state_dict(state_dict)


def _densenet(arch, growth_rate, block_config, num_init_features, pretrained, progress,
              **kwargs):
    model = DenseNet(growth_rate, block_config, num_init_features, **kwargs)
    model.network_name = arch
    if pretrained:
        _load_state_dict(model, model_urls[arch], progress)
    return model


def densenet18(pretrained=False, progress=True, **kwargs):
    r"""Densenet-18 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _densenet('densenet18', 32, (2, 2, 2, 2), 64, pretrained, progress,
                     **kwargs)


def densenet121(pretrained=False, progress=True, **kwargs):
    r"""Densenet-121 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _densenet('densenet121', 32, (6, 12, 24, 16), 64, pretrained, progress,
                     **kwargs)


def densenet161(pretrained=False, progress=True, **kwargs):
    r"""Densenet-161 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _densenet('densenet161', 48, (6, 12, 36, 24), 96, pretrained, progress,
                     **kwargs)


def densenet169(pretrained=False, progress=True, **kwargs):
    r"""Densenet-169 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _densenet('densenet169', 32, (6, 12, 32, 32), 64, pretrained, progress,
                     **kwargs)


def densenet201(pretrained=False, progress=True, **kwargs):
    r"""Densenet-201 model from
    `"Densely Connected Convolutional Networks" <https://arxiv.org/pdf/1608.06993.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _densenet('densenet201', 32, (6, 12, 48, 32), 64, pretrained, progress,
                     **kwargs)


# You are welcome to evaluate other CNN backbones
cnn = densenet18()

# feel free to modify these parameters
lstm_hidden_units = 128
lstm_layers = 1

# 0 means there is 0% probability of dropout happening
lstm_dropout = 0

model = CNNLSTMNetwork(cnn, lstm_hidden_units, lstm_layers, lstm_dropout)

# We highly recommend using SGD for this problem
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)
bce = nn.BCEWithLogitsLoss()


In [12]:
from sklearn.metrics import f1_score, accuracy_score
def perform_training_epoch(train_loader):
    model.train()
    train_loss = 0.0
    train_corrects = 0
    train_total = 0

    for data in train_loader:

        *_, y, z= data   # Unpack the inner tuple

        x = y.float().cuda()
        y = z.float().cuda()
        optimizer.zero_grad()  # Zero the gradients
        model.cuda()
        output = model(x)
        loss = bce(output, y)
        loss.backward()
        optimizer.step()

        # Update training loss
        train_loss += loss.item() * x.size(0)

        # Calculate training accuracy
        binary_predictions = (output > 0.5).float()
        train_corrects += torch.sum(binary_predictions == y.data)
        train_total += y.size(0)

    # Calculate training accuracy
    train_accuracy = train_corrects / train_total
    train_loss /= len(train_loader.dataset)

    print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")

    return train_loss, train_accuracy


def perform_inferencing(loader):
    model.eval()
    all_predictions = []
    all_targets = []
    total_loss = 0.0
    total_corrects = 0
    total_samples = 0

    with torch.no_grad():
        for _, y, z in loader:
            x = y.float().cuda()

            output = model(x)

            # Calculate loss
            loss = bce(output, z.float().cuda())
            total_loss += loss.item() * x.size(0)

            # Store predictions and ground truth labels
            all_predictions.append(output.cpu().numpy())
            all_targets.append(z.cpu().numpy())

            # Calculate accuracy
            binary_predictions = (output > 0.5).cpu().float()
            total_corrects += torch.sum(binary_predictions == z.data.cpu())
            total_samples += z.size(0)

    # Concatenate predictions and ground truth labels
    predictions = np.concatenate(all_predictions)
    targets = np.concatenate(all_targets)

    # Convert probabilities to binary predictions
    binary_predictions = (predictions > 0.5).astype(int)

    # Calculate F1 score and accuracy
    f1 = f1_score(targets, binary_predictions, average='macro')
    accuracy = accuracy_score(targets, binary_predictions)

    # Calculate average loss
    average_loss = total_loss / len(loader.dataset)

    print(f"F1 Score: {f1:.4f}, Accuracy: {accuracy:.4f}")
    return average_loss, accuracy


num_epochs = 5
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Training
    train_loss, train_accuracy = perform_training_epoch(train_loader)

    # Validation
    print("Validation Results")
    validation_loss, validation_accuracy = perform_inferencing(val_loader)
    print(f"Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_accuracy:.4f}")

# Test the model on the testing set
print("Testing Results")
test_loss, test_accuracy = perform_inferencing(test_loader)
print(f"Test Accuracy: {test_accuracy:.4f}")

Epoch 1/5
Training Loss: 0.6896, Training Accuracy: 1.0000
Validation Results
F1 Score: 0.0000, Accuracy: 0.0000
Validation Loss: 0.6910, Validation Accuracy: 0.0000
Epoch 2/5
Training Loss: 0.6794, Training Accuracy: 0.9999
Validation Results
F1 Score: 0.0004, Accuracy: 0.0002
Validation Loss: 0.6900, Validation Accuracy: 0.0002
Epoch 3/5
Training Loss: 0.6645, Training Accuracy: 1.0115
Validation Results
F1 Score: 0.0008, Accuracy: 0.0004
Validation Loss: 0.6878, Validation Accuracy: 0.0004
Epoch 4/5
Training Loss: 0.6313, Training Accuracy: 1.1313
Validation Results
F1 Score: 0.0008, Accuracy: 0.0004
Validation Loss: 0.6870, Validation Accuracy: 0.0004
Epoch 5/5
Training Loss: 0.5557, Training Accuracy: 1.2869
Validation Results
F1 Score: 0.3501, Accuracy: 0.2709
Validation Loss: 0.6764, Validation Accuracy: 0.2709
Testing Results
F1 Score: 0.4565, Accuracy: 0.4272
Test Accuracy: 0.4272
